
**Notes:**
Give the journey through the queries to get to the answer describing how we got there using weaviate queries

to tell a story around the application

Not the features 

this is how it can be used and its beneficial 


GenAi - Make it easier


Write what the story we want to do

## Get answers to USCG Auxiliary questions from authoritative sources.

ASK uses Artificial Intelligence (AI) to search over 251 Coast Guard Auxiliary references for answers. This is a working prototype for evaluation. Not an official USCG Auxiliary service. Learn more here.

ASK answers questions such as:

- What are the requirements to run for FC?
- How do I stay current in boat crew?
- ¿En que ocasiones es necesario un saludo militar?


Types of documents:  PDF manuals for Coast Gaurd Intelligence


The journey to better question/answer response
1. Keyword search
2. Semantic gets closer
3. Leverage GenAi 



### Example Query Types

#### Queries No Filters ####
- Keyword search on full document collection
- Semantic Search on full document collection
- Generative on Full document Collection
- Generative per result

#### Queries With Filters ####
- Time Range
- Source
- Organization






In [ ]:
import os

import re
import weaviate
import weaviate.classes as wvc
from dateutil import parser
from dotenv import load_dotenv, find_dotenv


load_dotenv(find_dotenv())

In [ ]:
print(os.environ.get("OPENAI_API_KEY"))

In [4]:
url = "https://fksy6wnuqfscrfox3d2c0w.c0.us-east1.gcp.weaviate.cloud"
api_key = "kpL3TO3LlGlD8D08nmDvLwhnQknZrvu64HO8"

client = weaviate.connect_to_wcs(
    cluster_url=url,
    auth_credentials=weaviate.auth.AuthApiKey(api_key),
    headers={
        "X-OpenAI-Api-Key": os.environ.get("OPENAI_API_KEY")
    }
)

### Check and see if Weviate is Healthy

In [ ]:
client.is_live()
import json

metainfo = client.get_meta()
print(json.dumps(metainfo, indent=2))  # Print the meta information in a readable format

# Query the Documents

In [6]:
COLLECTION_FULL_DOCUMENTS = "PDF_document"
COLLECTION_DOCUMENT_PAGES = "PDF_document_page"

QUERY = "What are the requirements to run for FC?"  

### Semantic Search (Near Text)

In [ ]:
import weaviate.classes.query as wq
import json
from weaviate.classes.query import QueryReference


# Get the collection
pdf_pages_collection = client.collections.get(COLLECTION_DOCUMENT_PAGES)

# Perform query to the 

response = pdf_pages_collection.query.near_text(
    query=QUERY, limit=5, 
    return_metadata=wq.MetadataQuery(distance=True),
    return_references=[
        QueryReference(
            link_on="hasPdfDocument",
            return_properties=["organization"]
        ),
    ]
)

# return_properties=["organization, curator, source, effective_date"]

# results = weaviate_response_to_dict(response)
# print(results)
print(response)
# # Inspect the response
for o in response.objects:
    print(json.dumps(o.properties, indent=2))  # Print the object properties (metadata and propert
    print("-------------------Response Metadata-------------------")
    print(o.metadata)
    for obj in o.references['hasPdfDocument'].objects:
        print(obj.properties)


### Semantic Search with Generative Output across all returned documents

##### Setup Prompt

In [ ]:
import pandas as pd

def retrieval_context_excel_to_dict(file_path):
    ''' Read Excel file into a dictionary of worksheets. 
    Each worksheet is its own dictionary. Column 1 is 
    the key. Column 2 is the values'''

    xls = pd.ExcelFile(file_path)
    dict = {}

    for sheet_name in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet_name)
        if df.shape[1] >= 2:
            dict[sheet_name] = pd.Series(
                df.iloc[:, 1].values, index=df.iloc[:, 0]).to_dict()
        else:
            print(f"The sheet '{sheet_name}' does not have enough columns.")
    return dict

retrieval_context_dict = retrieval_context_excel_to_dict('utils/retrieval_context.xlsx')
acronyms_dict = retrieval_context_dict.get("acronyms", None)
acronyms_json = json.dumps(acronyms_dict, indent=4)
terms_dict = retrieval_context_dict.get("terms", None)
terms_json = json.dumps(terms_dict, indent=4)

user_message = f"User question: {QUERY}```acronyms_json: {acronyms_json}\n\nterms_json: {terms_json}```"

query_prompt = f"""
Your task is to modify the user's question based on two lists: 'acronym_json' and 'terms_json'. Each list contains terms and their associated additional information. Follow these instructions:

- Review the user's question and identify if any acronyms from 'acronym_json' or phrases in 'terms_json' appear in it.
- If an acronym from 'acronym_json' replace the term with the associated additional information.
- If a phrase from 'terms_json' appears in the question, append its associated additional information to the end of the question.
- Do not remove or alter any other part of the original question.
- Do not provide an answer to the question.
- If no terms from either list are found in the question, leave the question as is.

Example:
- Question: How do I get a VE certification?
- Your response: How do I get a vessel examiner certification? Certification includes information about initial qualification.

- Question: What are the requirements for pilot training?
- Your response: What are the requirements for pilot training? Pilot is a position in the aviation program.

    - Question: What is required to stay current in the Auxiliary?
- Your response: What is required to stay current in the Auxiliary? To be in the Auxiliary, members are required to maintain the Core Training (AUXCT), undego an annual uniform inspection, and pay annual dues.

"""

### Create Instructions RAG Prompt

This is the second step as part of the QnA system

In [ ]:
instruction_prompt = f"""
    Use the following pieces of context to answer the users question. 
    
    INCLUDES ALL OF THE DETAILS IN YOUR RESPONSE, INDLUDING REQUIREMENTS AND REGULATIONS. 
    
    National Workshops are required for boat crew, aviation, and telecommunications when then are offered and you should mention this in questions about those programs. 
    Include Auxiliary Core Training (AUXCT) in your response for any question regarding certifications or officer positions.
    
    If you don't know the answer, just say I don't know, don't try to make up an answer. 
    """



#### Generated Response

A single prompt that uses all the results returned from weaviate to generate 1 response

In [ ]:
import weaviate.classes.query as wq
import json
from weaviate.classes.query import QueryReference
import textwrap



# Get the collection
pdf_pages_collection = client.collections.get(COLLECTION_DOCUMENT_PAGES)

# Perform query to the
# increase response window 

response = pdf_pages_collection.generate.near_text(
    query=QUERY, 
    limit=6,
    return_metadata=wq.MetadataQuery(distance=True),
    return_references=[
        QueryReference(
            link_on="hasPdfDocument",
            return_properties=["organization"]
        ),
    ],
    grouped_task=instruction_prompt,
    grouped_properties=["content"]

)

# return_properties=["organization, curator, source, effective_date"]


print(f"""
-----------------------------Generated Response------------------------------------------
{textwrap.fill(response.generated, width=80)}
-----------------------------End Generation-----------------------------\n
""")


 # Inspect the response
for o in response.objects:
    print(json.dumps(o.properties, indent=2))  # Print the object properties (metadata and propert
    print("-------------------Document  Metadata-------------------")
    print(o.metadata)
    for obj in o.references['hasPdfDocument'].objects:
        print(obj.properties)


### Single Response

A single prompt that will be applied to each individual document returned

In [ ]:
import weaviate.classes.query as wq
import json
from weaviate.classes.query import QueryReference
import textwrap



# Get the collection
pdf_pages_collection = client.collections.get(COLLECTION_DOCUMENT_PAGES)
prompt = "Summarize in 10 words"

# Perform query to the 

response = pdf_pages_collection.generate.near_text(
    query=QUERY, 
    limit=5,
    return_metadata=wq.MetadataQuery(distance=True),
    return_references=[
        QueryReference(
            link_on="hasPdfDocument",
            return_properties=["organization"]
        ),
    ],
    single_prompt=prompt
)

# return_properties=["organization, curator, source, effective_date"]



# # Inspect the response
for o in response.objects:
    print(json.dumps(o.properties, indent=2))  # Print the object properties (metadata and propert
    # -----------------------------Generated Response------------------------------------------
    # {textwrap.fill(response.generated, width=80)}
    print("-------------------Response Metadata-------------------")
    print(o.metadata)